- 70,000 data points
- 55,000 for training
- 10,000 for testing
- 5,000 for Validation

# Tensorflow Training
- Prepared Data -- MNIST Data 
- Interfrence -- sum(X* weight)+ Bias -> activation
- Loss Measurement -- Cross Entropy 
- Optimizer -- Adam Optimizer

Command to run TensorBoard Session ... type inside CMD

- cd/folder_name
- tensorboard --log ./tb_logs

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist= input_data.read_data_sets('MNIST_data/', one_hot= True)
mnist

W0804 12:55:02.522271  7212 deprecation.py:323] From <ipython-input-2-db594d3ab595>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0804 12:55:02.537886  7212 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0804 12:55:02.537886  7212 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implemen

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0804 12:55:02.722917  7212 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0804 12:55:02.722917  7212 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0804 12:55:02.769726  7212 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Plea

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000245A69C2898>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000245AD94E4E0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000245AD94E4A8>)

In [28]:
logPath= './tb_logs/'

In [13]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean= tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev= tf.sqrt(tf.reduce_mean(tf.square(var- mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

X is placeholder for the 28*28 image data

y_ or 'y Bar' and is a 10 element vector containing the digit(0-9) class. Such as [0.14, 0.8, 0, 0, 0, 0, 0, 0, 0.06].. right answers

In [5]:
with tf.name_scope('MNIST_Input'):
    X= tf.placeholder(tf.float32, shape= [None, 784], name= 'X')
    y_= tf.placeholder(tf.float32, [None,10], name='y_')

Change the MNIST input data from a list of values to a 28 pixel X 28 pixel X 1 grayscale value cube which the Convolution NN can use.

In [6]:
with tf.name_scope("Input_Reshape"):
    X_image= tf.reshape(X, [-1, 28, 28, 1], name= 'x_image')
    tf.summary.image('input_img', X_image, 5)

Define helper functions to created weights and baises variables, and convolution, and pooling layers. We are using RELU as our activation function.  These must be initialized to a small positive number and with some noise so you don't end up going to zero when comparing diffs

In [7]:
def weight_variable(shape, name=None):
    initial= tf.truncated_normal(shape, stddev= 0.1)
    return tf.Variable(initial)

In [8]:
def bias_variable(shape, name=None):
    initial= tf.constant(0.1, shape= shape)
    return tf.Variable(initial)

Convolution and Pooling - we do Convolution, and then pooling to control overfitting

In [9]:
def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides= [1, 1, 1, 1], padding= 'SAME')

In [10]:
def max_pool_2x2(x, name=None):
    return tf.nn.max_pool(x, ksize= [1, 2, 2, 1],
                         strides= [1, 2, 2, 1], padding= 'SAME')

# Define layers in the NN

1st Convolution layer
- 32 features for each 5X5 patch of the image

Do convolution on images, add bias and push through RELU activation

Take results and run through max_pool

In [14]:
with tf.name_scope('Conv1'):
    with tf.name_scope('weights'):
        W_conv1= weight_variable([5, 5, 1, 32])
    with tf.name_scope('biases'):
        b_conv1= bias_variable([32])
        variable_summaries(b_conv1)
        
    # Do convolution on images, add bias and push through RELU activation
    conv1_wx_b = conv2d(X_image, W_conv1,name="conv2d") + b_conv1
    tf.summary.histogram('conv1_wx_b', conv1_wx_b)
    h_conv1 = tf.nn.relu(conv1_wx_b, name="relu")
    tf.summary.histogram('h_conv1', h_conv1)
    # take results and run through max_pool
    h_pool1 = max_pool_2x2(h_conv1, name="pool")

2nd Convolution layer
- Process the 32 features from Convolution layer 1, in 5 X 5 patch.  Return 64 features weights and biases

Do convolution of the output of the 1st convolution layer.  Pool results 

In [15]:
with tf.name_scope('Conv2'):
# Process the 32 features from Convolution layer 1, in 5 X 5 patch.  return 64 features weights and biases
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5, 5, 32, 64], name="weight")
        variable_summaries(W_conv2)
    with tf.name_scope('biases'):
        b_conv2 = bias_variable([64], name="bias")
        variable_summaries(b_conv2)
    # Do convolution of the output of the 1st convolution layer.  Pool results 
    conv2_wx_b = conv2d(h_pool1, W_conv2, name="conv2d") + b_conv2
    tf.summary.histogram('conv2_wx_b', conv2_wx_b)
    h_conv2 = tf.nn.relu(conv2_wx_b, name="relu")
    tf.summary.histogram('h_conv2', h_conv2)
    h_pool2 = max_pool_2x2(h_conv2, name="pool")

Fully Connected Layer

Connect output of pooling layer 2 as input to full connected layer

In [16]:
with tf.name_scope('FC'):
    # Fully Connected Layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024], name="weight")
    b_fc1 = bias_variable([1024], name="bias")
    #   Connect output of pooling layer 2 as input to full connected layer
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="relu")

Dropout Some Neurons to reduce OverFitting

In [17]:
keep_prob = tf.placeholder(tf.float32)  # get dropout probability as a training input.
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W0804 12:57:05.279460  7212 deprecation.py:506] From <ipython-input-17-a82f30c66d70>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Readout layer.. to get 10 digits 0-9

In [18]:
with tf.name_scope("Readout"):
# Readout layer
    W_fc2 = weight_variable([1024, 10], name="weight")
    b_fc2 = bias_variable([10], name="bias")

# Define Model

In [19]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

Loss Measurement

In [20]:
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

W0804 12:57:08.863332  7212 deprecation.py:323] From <ipython-input-20-2a4a614328ad>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Loss optimization

In [21]:
with tf.name_scope("loss_optimizer"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [22]:
with tf.name_scope("accuracy"):
    # What is correct
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    # How accurate is it?
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
tf.summary.scalar("cross_entropy_scl", cross_entropy)
tf.summary.scalar("training_accuracy", accuracy)

<tf.Tensor 'training_accuracy:0' shape=() dtype=string>

In [24]:
# TB - Merge summaries 
summarize_all = tf.summary.merge_all()

To run Tensorflow on CPU

In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

Initialize all of the variables

In [26]:
sess= tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [29]:
# TB - Write the default graph out so we can view it's structure
tbWriter = tf.summary.FileWriter(logPath, sess.graph)

# Train Model

In [30]:
import time

num_steps= 2000
display_every= 100

In [31]:
# Start timer
start_time = time.time()

For 1000 iters and Displaying Result after every 100 Iterations

In [32]:
for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    _, summary = sess.run([train_step, summarize_all], feed_dict={X: batch[0], y_: batch[1], keep_prob: 0.5})


    # Periodic status display
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={
            X:batch[0], y_: batch[1], keep_prob: 1.0})
        end_time = time.time()
        print("step {0}, elapsed time {1:.2f} seconds, training accuracy {2:.3f}%".format(i, end_time-start_time, train_accuracy*100.0))
        # write summary to log
        tbWriter.add_summary(summary,i)

step 0, elapsed time 2.27 seconds, training accuracy 2.000%
step 100, elapsed time 10.59 seconds, training accuracy 76.000%
step 200, elapsed time 19.14 seconds, training accuracy 94.000%
step 300, elapsed time 28.04 seconds, training accuracy 94.000%
step 400, elapsed time 37.74 seconds, training accuracy 84.000%
step 500, elapsed time 47.57 seconds, training accuracy 94.000%
step 600, elapsed time 57.58 seconds, training accuracy 94.000%
step 700, elapsed time 67.58 seconds, training accuracy 84.000%
step 800, elapsed time 77.75 seconds, training accuracy 98.000%
step 900, elapsed time 88.02 seconds, training accuracy 98.000%
step 1000, elapsed time 98.23 seconds, training accuracy 92.000%
step 1100, elapsed time 108.82 seconds, training accuracy 98.000%
step 1200, elapsed time 119.28 seconds, training accuracy 98.000%
step 1300, elapsed time 129.87 seconds, training accuracy 98.000%
step 1400, elapsed time 140.61 seconds, training accuracy 96.000%
step 1500, elapsed time 151.28 seco

Evaluate how well the model did. Do this by comparing the digit with the highest probability in actual (y) and predicted (y_).

In [33]:
# Display summary 
#     Time to train
end_time = time.time()
print("Total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))



Total training time for 2000 batches: 208.00 seconds


In [34]:
#     Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={
    X: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})*100.0))

sess.close()

Test accuracy 97.760%
